In [1]:
import yt
import numpy as np
from matplotlib import pylab
from yt.analysis_modules.halo_analysis.api import HaloCatalog

/storage/home/hcoda1/0/tnguyen747/.conda/envs/fairytienbase/lib/python3.7/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: Development of the HaloCatalog module has been moved to the yt_astro_analysis package. This version is deprecated and will be removed from yt in a future release. See https://github.com/yt-project/yt_astro_analysis for further information.


In [2]:
from yt.funcs import mylog

# only critical log messages are printed
mylog.setLevel(50)

# the default log level is 20, which shows warnings and info messages
# mylog.setLevel(20)

In [31]:
ts = yt.load('~jw254/data/SG64-2020/DD????/output_????')
len(ts)

125

In [32]:
halos_ts = yt.load('~jw254/data/SG64-2020/rockstar_halos-jhw/halos_DD????.0.bin')
len(halos_ts)

125

In [33]:
for ds in ts:
    print(ds.current_time.in_units('Gyr'))

0.004369889293125582 Gyr
0.013758638303994366 Gyr
0.01845300978238316 Gyr
0.023147397943200813 Gyr
0.027841793315125374 Gyr
0.03253616155572588 Gyr
0.03723055260165262 Gyr
0.041924965679706314 Gyr
0.046619400045454285 Gyr
0.05131374284518717 Gyr
0.05600820907733894 Gyr
0.060702563165064345 Gyr
0.06539691726491183 Gyr
0.07009127136490917 Gyr
0.07478579034126517 Gyr
0.07948015546656982 Gyr
0.08417452060808006 Gyr
0.08886888574970805 Gyr
0.09356325089146447 Gyr
0.09825761603336004 Gyr
0.10295198117540012 Gyr
0.10764634631759006 Gyr
0.11234095910113515 Gyr
0.11703533501703901 Gyr
0.12172971095725128 Gyr
0.1264240868975438 Gyr
0.13111846283792733 Gyr
0.1358128387784018 Gyr
0.14050721471897265 Gyr
0.14520159065964516 Gyr
0.14989596660041937 Gyr
0.15459034254130596 Gyr
0.15928481760619012 Gyr
0.16397921752356603 Gyr
0.16867359773827523 Gyr
0.17336777812863186 Gyr
0.1780621679025885 Gyr
0.18076616079106833 Gyr
0.18275676838217428 Gyr
0.1874511346570214 Gyr
0.19214553310671045 Gyr
0.19683991937

# BH particle filters

In [17]:
@yt.particle_filter('p3_sn', ['particle_mass', 'particle_type'])
def p3_sn(pfilter, data):
    return (data['particle_type'] == 5) & (data['particle_mass'].in_units('Msun') < 1e-10)

@yt.particle_filter('p3_bh', ['creation_time', 'particle_mass', 'particle_type'])
def p3_bh(pfilter, data):
    return (data['particle_type'] == 1) & (data['creation_time'] > 0) & \
        (data['particle_mass'].in_units('Msun') > 1e-3)

@yt.particle_filter('p3_living', ['particle_mass', 'particle_type'])
def p3_living(pfilter, data):
    return (data['particle_type'] == 5) & (data['particle_mass'].in_units('Msun') > 1e-3)

@yt.particle_filter('p3', ['particle_mass', 'particle_type', 'creation_time'])
def p3(pfilter, data):
    return ((data['particle_type'] == 5) & (data['particle_mass'].in_units('Msun') < 1e-10)) \
        | ((data['particle_type'] == 1) & (data['creation_time'] > 0) & \
           (data['particle_mass'].in_units('Msun') > 1)) \
        | ((data['particle_type'] == 5) & (data['particle_mass'].in_units('Msun') > 1e-3))

@yt.particle_filter('p2', ['particle_mass', 'particle_type'])
def p2(pfilter, data):
    return (data['particle_type'] == 7)

# Automate finding the limits of colorbar

In [6]:
mins, maxs = [], []
for j in range(len(ts)):
    ds = ts[j] # MUST save to another variable for the particle filter to work
    ds.add_particle_filter('p3_bh')
    prj = yt.ProjectionPlot(ds, 'x', 'density', weight_field='density', width = (5, 'kpc'))
    # automate finding limits of colorbar
    mins.append(prj['density'].cb.vmin)
    maxs.append(prj['density'].cb.vmax)
print(min(mins), max(maxs))

Parsing Hierarchy : 100%|██████████| 777/777 [00:00<00:00, 8728.77it/s]


1.3373855651996533e-27 6.1912356234886716e-21


In [8]:
# stretch the limits
print(min(mins)*0.9)
print(max(maxs)*1.1)

1.203647008679688e-27
6.81035918583754e-21


# Making and saving projections with fixed colorbar limits & top 10 halo indicators

In [34]:
for j in range(len(ts)):
    ds = ts[j] # MUST save to another variable for the particle filter to work
    ds.add_particle_filter('p3_bh')
    #ad = ds.all_data() # using ts[j] do NOT work because the particle filter isn't saved
    #bh_id = ad['p3_bh', 'particle_index']
    prj = yt.ProjectionPlot(ds, 'x', 'density', weight_field='density', width = (5, 'kpc'))
    prj.annotate_particles((5, 'kpc'), p_size=10, ptype='p3_bh', col='red')
    
# annotate top 10 halos
    # contain halos_ts[j] in a variable to avoid weak-reference ERROR
    hds = halos_ts[j]
    # create initial halo catalog
    hc = HaloCatalog(data_ds=ds, halos_ds=hds) # need to create a HaloCatalog object to add filters
    # to get the top 10 halos, first, convert hds yt_array to numpy_array
    halos_np = np.array(hds.r['particle_mass'].in_units('Msun'))
    # sort through mass and return indices
    sorted_id = np.argsort(halos_np) # increasing order
    
    # hc could have 0 halos and the filtered hc will NOT work --> create 2 conditions
    if len(sorted_id) > 0:
        # remember the 10th (or the lowest in the list) largest value
        filter_value = halos_np[sorted_id][-10:][0]
        # add_filter to filter from the 10th largest (== the lowest in the list) value
        hc.add_filter('quantity_value', 'particle_mass', '>=', filter_value, 'Msun')
        # create filtered catalog
        hc.create()
        # load the filtered catalog
        halos_filtered = yt.load("halo_catalogs/catalog/catalog.0.h5")
    
    else: # if the initial hc has 0 halos
        halos_filtered = hc # let filtered halo catalog be the initial hc
        
    # annotate halos
    prj.annotate_halos(halos_filtered)
    
# timestamps & set limits for colorbar
    prj.annotate_timestamp(redshift=True) # add timestamp and redshift
    prj.set_zlim('density', zmin=1.3e-27, zmax=1e-23) # set limits for colorbar
    prj.save()

Creating catalog:   0%|          | 0/264 [00:00<?, ?it/s]